# Using BigQuery to perform basic data analytics

Here is a notebook to execute some SQL commands in order to analyse and then visualise data from the Austin Bike Share Trips dataset available in Google BigQuery.

In [10]:
# Library Imports
import os
import pandas as pd

#Imports google cloud client library
from google.cloud import bigquery

In [11]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [12]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './atomic-arbor-367319-4fe051d91661.json'

## Austin Bike Share Dataset

 There are 2 tables in this dataset:
  - Bikeshare stations
  - Bikeshare Trips

In [13]:
bikeshare_stations = 'bigquery-public-data.austin_bikeshare.bikeshare_stations'
bikeshare_trips = 'bigquery-public-data.austin_bikeshare.bikeshare_trips'

In [14]:
# initiates BQ service
bigquery_client = bigquery.Client()

In [19]:
bigquery_client.get_table(bikeshare_stations).schema

[SchemaField('station_id', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('name', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('status', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('address', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('alternate_name', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('city_asset_number', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('property_type', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('number_of_docks', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('power_type', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('footprint_length', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('footprint_width', 'FLOAT', 'NULLABLE', None, (), None),
 SchemaField('notes', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('council_district', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('modified_date', 'TIMESTAMP', 'NULLABLE', None, (), None)]

In [20]:
bigquery_client.get_table(bikeshare_trips).schema

[SchemaField('trip_id', 'INTEGER', 'NULLABLE', 'Numeric ID of bike trip', (), None),
 SchemaField('subscriber_type', 'STRING', 'NULLABLE', 'Type of the Subscriber', (), None),
 SchemaField('bikeid', 'STRING', 'NULLABLE', 'ID of bike used', (), None),
 SchemaField('start_time', 'TIMESTAMP', 'NULLABLE', 'Start timestamp of trip', (), None),
 SchemaField('start_station_id', 'INTEGER', 'NULLABLE', 'Numeric reference for start station', (), None),
 SchemaField('start_station_name', 'STRING', 'NULLABLE', 'Station name for start station', (), None),
 SchemaField('end_station_id', 'STRING', 'NULLABLE', 'Numeric reference for end station', (), None),
 SchemaField('end_station_name', 'STRING', 'NULLABLE', 'Station name for end station', (), None),
 SchemaField('duration_minutes', 'INTEGER', 'NULLABLE', 'Time of trip in minutes', (), None)]

In [21]:
QUERY = f'''
SELECT * FROM `{bikeshare_stations}`
LIMIT 5
'''

#Run the query and write result to a pandas data frame
Query_Results = bigquery_client.query(QUERY)
bikeshare_stations_df = Query_Results.to_dataframe()
#View top few rows of result
bikeshare_stations_df

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2538,Bullock Museum @ Congress & MLK,closed,1881 Congress Ave.,None,<NA>,None,<NA>,None,<NA>,NaN,None,1,2021-01-04 12:00:00+00:00
1,2541,State Capitol @ 14th & Colorado,closed,206 W. 14th St.,None,<NA>,None,<NA>,None,<NA>,NaN,None,1,2021-01-04 12:00:00+00:00
2,2545,ACC - Rio Grande & 12th,closed,700 W. 12th St.,None,<NA>,None,<NA>,None,<NA>,NaN,None,9,2021-01-04 12:00:00+00:00
3,2546,ACC - West & 12th Street,closed,1231 West Ave.,None,<NA>,None,<NA>,None,<NA>,NaN,None,9,2021-01-04 12:00:00+00:00
4,2564,5th & San Marcos,closed,991 East 5th,None,<NA>,None,<NA>,None,<NA>,NaN,None,3,2021-01-04 12:00:00+00:00


In [22]:
QUERY = f'''
SELECT * FROM `{bikeshare_trips}`
LIMIT 5
'''

#Run the query and write result to a pandas data frame
Query_Results = bigquery_client.query(QUERY)
bikeshare_trips_df = Query_Results.to_dataframe()
#View top few rows of result
bikeshare_trips_df

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900285908,Annual Membership (Austin B-cycle),400,2014-10-26 14:12:00+00:00,2823,Capital Metro HQ - East 5th at Broadway,2544,East 6th & Pedernales St.,10
1,9900289692,Walk Up,248,2015-10-02 21:12:01+00:00,1006,Zilker Park West,1008,Nueces @ 3rd,39
2,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
3,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
4,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30


Can also use the `%%bigquery` magic command

In [17]:
%%bigquery
SELECT * FROM bigquery-public-data.austin_bikeshare.bikeshare_trips
LIMIT 3

Downloading: 100%|██████████| 3/3 [00:01<00:00,  2.18rows/s]


,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900285908,Annual Membership (Austin B-cycle),400,2014-10-26 14:12:00+00:00,2823,Capital Metro HQ - East 5th at Broadway,2544,East 6th & Pedernales St.,10
1,9900289692,Walk Up,248,2015-10-02 21:12:01+00:00,1006,Zilker Park West,1008,Nueces @ 3rd,39
2,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31


In [26]:
%%bigquery
SELECT COUNT(*) AS total_trips FROM bigquery-public-data.austin_bikeshare.bikeshare_trips

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.66s/rows]


,total_trips
0,1694087


In [27]:
%%bigquery
SELECT COUNT(*) AS total_stations FROM bigquery-public-data.austin_bikeshare.bikeshare_stations

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.38s/rows]


,total_stations
0,102


In [35]:
%%bigquery
SELECT 
    SUM(CASE WHEN trip_id is null THEN 1 ELSE 0 END) AS trip_id_nulls,
    SUM(CASE WHEN subscriber_type is null THEN 1 ELSE 0 END) AS subscriber_type_nulls,
    SUM(CASE WHEN bikeid is null THEN 1 ELSE 0 END) AS bikeid_nulls,
    SUM(CASE WHEN start_time is null THEN 1 ELSE 0 END) AS start_time_nulls,
    SUM(CASE WHEN start_station_id is null THEN 1 ELSE 0 END) AS start_station_id_nulls,
    SUM(CASE WHEN start_station_name is null THEN 1 ELSE 0 END) AS start_station_name_nulls,
    SUM(CASE WHEN end_station_id is null THEN 1 ELSE 0 END) AS end_station_id_nulls,
    SUM(CASE WHEN end_station_name is null THEN 1 ELSE 0 END) AS end_station_name_nulls,
    SUM(CASE WHEN duration_minutes is null THEN 1 ELSE 0 END) AS duration_minutes_nulls,
FROM bigquery-public-data.austin_bikeshare.bikeshare_trips

Downloading:   0%|          | 0/1 [00:00<?, ?rows/s]

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.54s/rows]


,trip_id_nulls,subscriber_type_nulls,bikeid_nulls,start_time_nulls,start_station_id_nulls,start_station_name_nulls,end_station_id_nulls,end_station_name_nulls,duration_minutes_nulls
0,0,6569,723,0,23186,0,24453,0,0


In [38]:
%%bigquery
SELECT
    SUM(CASE WHEN station_id is null THEN 1 ELSE 0 END) AS station_id_nulls,
    SUM(CASE WHEN name is null THEN 1 ELSE 0 END) AS name_nulls,
    SUM(CASE WHEN status is null THEN 1 ELSE 0 END) AS status_nulls,
    SUM(CASE WHEN address is null THEN 1 ELSE 0 END) AS address_nulls,
    SUM(CASE WHEN alternate_name is null THEN 1 ELSE 0 END) AS alternate_name_nulls,
    SUM(CASE WHEN city_asset_number is null THEN 1 ELSE 0 END) AS city_asset_number_nulls,
    SUM(CASE WHEN property_type is null THEN 1 ELSE 0 END) AS property_type_nulls,
    SUM(CASE WHEN number_of_docks is null THEN 1 ELSE 0 END) AS number_of_docks_nulls,
    SUM(CASE WHEN power_type is null THEN 1 ELSE 0 END) AS power_type_nulls,
    SUM(CASE WHEN footprint_length is null THEN 1 ELSE 0 END) AS footprint_length_nulls,
    SUM(CASE WHEN footprint_width is null THEN 1 ELSE 0 END) AS footprint_width_nulls,
    SUM(CASE WHEN notes is null THEN 1 ELSE 0 END) AS notes_nulls,
    SUM(CASE WHEN council_district is null THEN 1 ELSE 0 END) AS council_district_nulls,
    SUM(CASE WHEN modified_date is null THEN 1 ELSE 0 END) AS modified_date_nulls,
FROM bigquery-public-data.austin_bikeshare.bikeshare_stations

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.57s/rows]


,station_id_nulls,name_nulls,status_nulls,address_nulls,alternate_name_nulls,city_asset_number_nulls,property_type_nulls,number_of_docks_nulls,power_type_nulls,footprint_length_nulls,footprint_width_nulls,notes_nulls,council_district_nulls,modified_date_nulls
0,0,0,0,1,100,25,18,20,18,23,23,70,0,0


In [47]:
%%bigquery
SELECT council_district, COUNT(*) AS number_bike_stations
FROM bigquery-public-data.austin_bikeshare.bikeshare_stations
GROUP BY council_district
ORDER BY number_bike_stations DESC

Downloading: 100%|██████████| 6/6 [00:01<00:00,  3.89rows/s]


,council_district,number_bike_stations
0,9,60
1,1,16
2,3,16
3,5,5
4,8,3
5,10,2


In [48]:
%%bigquery
SELECT DISTINCT subscriber_type
FROM bigquery-public-data.austin_bikeshare.bikeshare_trips

Downloading: 100%|██████████| 77/77 [00:01<00:00, 46.03rows/s]


,subscriber_type
0,Annual Membership (Austin B-cycle)
1,Walk Up
2,24-Hour Kiosk (Austin B-cycle)
3,Founding Member (Austin B-cycle)
4,Local365
...,...
72,PROHIBITED
73,RESTRICTED
74,Annual (Omaha B-cycle)
75,Local365 Youth (age 13-17 riders)- 1/2 off Spe...


In [51]:
%%bigquery
SELECT subscriber_type, COUNT(*) AS num_trips
FROM bigquery-public-data.austin_bikeshare.bikeshare_trips
GROUP BY subscriber_type
ORDER BY num_trips DESC
LIMIT 20

Downloading: 100%|██████████| 20/20 [00:01<00:00, 12.90rows/s]


,subscriber_type,num_trips
0,Walk Up,374151
1,Local365,298566
2,U.T. Student Membership,291978
3,Student Membership,118200
4,24-Hour Kiosk (Austin B-cycle),108672
5,Local30,69062
6,Local31,65792
7,Explorer,60565
8,24 Hour Walk Up Pass,50994
9,Weekender,45012


In [59]:
%%bigquery
SELECT start_station_id, start_station_name, COUNT(*) AS num_trips
FROM bigquery-public-data.austin_bikeshare.bikeshare_trips
GROUP BY start_station_name, start_station_id
ORDER BY num_trips DESC
LIMIT 8

Downloading: 100%|██████████| 8/8 [00:01<00:00,  5.12rows/s]


,start_station_id,start_station_name,num_trips
0,3798,21st & Speedway @PCL,72799
1,2575,Riverside @ S. Lamar,40635
2,3798,21st/Speedway @ PCL,39353
3,2499,City Hall / Lavaca & 2nd,36520
4,2494,2nd & Congress,35307
5,2707,Rainey St @ Cummings,34758
6,2501,5th & Bowie,34283
7,2574,Zilker Park,32640


In [72]:
%%bigquery
SELECT property_type, COUNT(*) AS num_stations
FROM bigquery-public-data.austin_bikeshare.bikeshare_stations
GROUP BY property_type, status
HAVING status = 'active'
ORDER BY num_stations DESC

Downloading: 100%|██████████| 5/5 [00:01<00:00,  3.84rows/s]


,property_type,num_stations
0,sidewalk,28
1,undetermined_parking,19
2,paid_parking,17
3,nonmetered_parking,8
4,parkland,6


In [65]:
%%bigquery
SELECT * FROM bigquery-public-data.austin_bikeshare.bikeshare_stations
WHERE status = 'active'
ORDER BY number_of_docks DESC

Downloading: 100%|██████████| 78/78 [00:01<00:00, 51.51rows/s]


,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,3798,21st & Speedway @PCL,active,E. 21st St.,None,32675,sidewalk,22,solar,30,7.5,On UT property-2 sided,9,2021-01-04 12:00:00+00:00
1,3799,23rd & San Jacinto @ DKR Stadium,active,E. 23rd St.,None,32676,sidewalk,22,solar,30,7.5,On UT property-2 sided,1,2021-01-04 12:00:00+00:00
2,2566,Pfluger Bridge @ W 2nd Street,active,Presented by Austin Parks Foundation,None,16762,sidewalk,19,solar,55,5.0,adjacent to parkland,9,2021-01-04 12:00:00+00:00
3,2499,City Hall / Lavaca & 2nd,active,Presented by Graves Dougherty Hearon & Moody,None,16732,sidewalk,19,solar,55,5.0,check dock #-18?,9,2021-01-04 12:00:00+00:00
4,3377,MoPac Pedestrian Bridge @ Veterans Drive,active,2105 Veterans Dr.,None,16598,undetermined_parking,19,solar,55,5.0,parkland in ROW/eastment,9,2021-01-04 12:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,3687,Boardwalk West,active,300 E. Riverside Dr.,None,16683,parkland,9,solar,30,5.0,None,9,2021-01-04 12:00:00+00:00
74,4047,8th & Lavaca,active,750 Lavaca Street,None,32710,sidewalk,9,solar,25,5.0,None,9,2021-01-04 12:00:00+00:00
75,2503,South Congress & James,active,Presented by The Austin Chronicle,None,16730,sidewalk,9,solar,26,7.5,on bulb-out,9,2021-01-04 12:00:00+00:00
76,3292,East 4th & Chicon,active,1819 East 4th St.,None,32516,undetermined_parking,9,solar,30,5.0,None,3,2021-01-04 12:00:00+00:00


In [55]:
%%bigquery
SELECT end_station_name, COUNT(*) AS num_trips
FROM bigquery-public-data.austin_bikeshare.bikeshare_trips
GROUP BY end_station_name
ORDER BY num_trips DESC
LIMIT 8

Downloading:   0%|          | 0/8 [00:00<?, ?rows/s]

Downloading: 100%|██████████| 8/8 [00:01<00:00,  5.06rows/s]


,end_station_name,num_trips
0,21st & Speedway @PCL,75055
1,City Hall / Lavaca & 2nd,41995
2,21st/Speedway @ PCL,40366
3,Riverside @ S. Lamar,39620
4,2nd & Congress,38842
5,4th & Congress,35240
6,Zilker Park,34898
7,Convention Center / 4th St. @ MetroRail,34010


In [57]:
%%bigquery
SELECT start_station_name, end_station_name, COUNT(*) AS num_trips
FROM bigquery-public-data.austin_bikeshare.bikeshare_trips
GROUP BY start_station_name, end_station_name
ORDER BY num_trips DESC
LIMIT 8

Downloading: 100%|██████████| 8/8 [00:01<00:00,  4.40rows/s]


,start_station_name,end_station_name,num_trips
0,21st & Speedway @PCL,Dean Keeton & Speedway,13737
1,21st & Speedway @PCL,21st & Speedway @PCL,13109
2,Dean Keeton & Speedway,21st & Speedway @PCL,13068
3,Riverside @ S. Lamar,Riverside @ S. Lamar,12302
4,Zilker Park,Zilker Park,10918
5,Rainey St @ Cummings,Rainey St @ Cummings,8676
6,Dean Keeton & Whitis,21st & Speedway @PCL,8598
7,21st & Speedway @PCL,Dean Keeton & Whitis,7941


In [74]:
%%bigquery
SELECT trips.*, 
stations.property_type AS start_station_property_type,
stations.number_of_docks AS start_station_number_of_docks,
stations.council_district AS start_station_council_district,
FROM bigquery-public-data.austin_bikeshare.bikeshare_trips trips
INNER JOIN bigquery-public-data.austin_bikeshare.bikeshare_stations stations
ON trips.start_station_id = stations.station_id

Downloading:   0%|          | 0/1650419 [00:00<?, ?rows/s]

Downloading: 100%|██████████| 1650419/1650419 [00:04<00:00, 341592.45rows/s]


,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes,start_station_property_type,start_station_number_of_docks,start_station_council_district
0,9900285908,Annual Membership (Austin B-cycle),400,2014-10-26 14:12:00+00:00,2823,Capital Metro HQ - East 5th at Broadway,2544,East 6th & Pedernales St.,10,undetermined_parking,13,3
1,9900289692,Walk Up,248,2015-10-02 21:12:01+00:00,1006,Zilker Park West,1008,Nueces @ 3rd,39,None,<NA>,8
2,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31,None,<NA>,5
3,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31,None,<NA>,5
4,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30,None,<NA>,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1650414,23557062,Local31,19254,2021-02-27 14:20:08+00:00,3838,26th/Nueces,4062,Lakeshore/Pleasant Valley,28,undetermined_parking,13,9
1650415,25272634,Local31,19453,2021-09-29 15:03:38+00:00,3838,26th/Nueces,4062,Lakeshore/Pleasant Valley,62,undetermined_parking,13,9
1650416,25179337,Student Membership,21829,2021-09-19 09:16:12+00:00,3838,26th/Nueces,4062,Lakeshore/Pleasant Valley,22,undetermined_parking,13,9
1650417,25150008,Local31,21859,2021-09-16 15:34:42+00:00,3838,26th/Nueces,4062,Lakeshore/Pleasant Valley,34,undetermined_parking,13,9


In [81]:
%%bigquery
SELECT *, EXTRACT(MONTH FROM start_time) AS month,
EXTRACT(DAYOFWEEK FROM start_time) AS day_of_week,
EXTRACT(HOUR FROM start_time) AS hour,
(EXTRACT(DAYOFWEEK FROM start_time)<2 OR EXTRACT(DAYOFWEEK FROM start_time)>6) AS is_weekend
FROM bigquery-public-data.austin_bikeshare.bikeshare_trips

Downloading: 100%|██████████| 1694087/1694087 [00:04<00:00, 406845.52rows/s]


,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes,month,day_of_week,hour,is_weekend
0,9900285908,Annual Membership (Austin B-cycle),400,2014-10-26 14:12:00+00:00,2823,Capital Metro HQ - East 5th at Broadway,2544,East 6th & Pedernales St.,10,10,1,14,True
1,9900289692,Walk Up,248,2015-10-02 21:12:01+00:00,1006,Zilker Park West,1008,Nueces @ 3rd,39,10,6,21,False
2,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31,10,1,15,True
3,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31,10,1,15,True
4,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30,10,1,15,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1694082,23557062,Local31,19254,2021-02-27 14:20:08+00:00,3838,26th/Nueces,4062,Lakeshore/Pleasant Valley,28,2,7,14,True
1694083,25272634,Local31,19453,2021-09-29 15:03:38+00:00,3838,26th/Nueces,4062,Lakeshore/Pleasant Valley,62,9,4,15,False
1694084,25179337,Student Membership,21829,2021-09-19 09:16:12+00:00,3838,26th/Nueces,4062,Lakeshore/Pleasant Valley,22,9,1,9,True
1694085,25150008,Local31,21859,2021-09-16 15:34:42+00:00,3838,26th/Nueces,4062,Lakeshore/Pleasant Valley,34,9,5,15,False
